In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!unzip /content/drive/MyDrive/cs4100_data/spotify_million_playlist_dataset.zip
!pip install ijson

In [ ]:
import ijson
import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
files = glob.glob("data/mpd.slice.*.json")


playlists = []
songs_lst = []
i = 0
for filename in tqdm(files):

  with open(filename, 'r') as f:
    objects = ijson.items(f, 'playlists.item')
    columns = list(objects)

    column_names = [col["tracks"] for col in columns]
    
    track_features_uri = []
    track_features_name = []
    track_features_artist_name = []
    track_features_artist_uri = []
    for playlist in column_names:
      for track in playlist:
        track_features_uri.append(track['track_uri'][14:])
        track_features_name.append(track['track_name'])
        track_features_artist_name.append(track['artist_name'])
        track_features_artist_uri.append(track['artist_uri'])
    
    track_features = pd.DataFrame(zip(track_features_name, track_features_artist_name, track_features_artist_uri), columns=['track_name','artist_name','artist_uri'])
    track_features["ids"] = track_features_uri
    track_features = track_features.drop_duplicates()
    track_features = track_features.reset_index(drop=True)
    songs_lst.append(track_features)
    playlists += [[track["track_uri"][14:] for track in p] for p in column_names]
    if len(songs_lst) > 50:
      new_df = pd.concat(songs_lst,ignore_index=True)
      new_df = new_df.set_index('ids').drop_duplicates().reset_index()
      songs_lst = [new_df]

    if len(playlists) >= 100000:
      with open('playlists_{}.npy'.format(i), 'wb') as f:
        np.save(f, np.array(playlists))
      i += 1
      playlists = []
songs = pd.concat(songs_lst,ignore_index=True).set_index('ids').drop_duplicates().reset_index()
print(songs.shape)
print(songs.ids.nunique())
songs.head()


In [ ]:
len(playlists)

In [ ]:
songs = songs[['track_name', 'artist_name', 'artist_name', 'artist_uri', 'ids']]

In [ ]:
songs.shape

In [ ]:
songs.to_csv('songs.csv')
